<a href="https://colab.research.google.com/github/gmelaku/Assignment1/blob/master/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework-5: Data Cleaning
## Adapted from: https://github.com/XD-DENG/Spark-practice
## 1: Please upload "2015-12-12.csv" (provided at the beggining of the HW-5 instructions) inside a "sample_data" directory
## 2: Fill out right hand side below each # TODO comments in the code. 

If you do not have spark-hadoop setup properly on your system, use follwing lines of code to download and setup:


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# !tar -xvf /content/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf  spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

# Create a blank SparkContext
from pyspark import SparkContext
sc = SparkContext()

# Upload the csv file into the notebook

In [7]:
from google.colab import files
uploaded = files.upload()

Saving 2015-12-12.csv to 2015-12-12.csv


## 1] Upload a csv file into a RDD and print how many entries are there

In [9]:
#TODO: Load the file
raw_content = sc.textFile("2015-12-12.csv")
print(type(raw_content))
# TODO print total entries in the RDD
raw_content.count()

<class 'pyspark.rdd.RDD'>


421970

### 2] Use take() and takeSample() on the loaded RDD content in order to check the difference between both calls

In [10]:
# TODO: use take() for getting 5 rows

raw_content.take(5)

[u'"date","time","size","r_version","r_arch","r_os","package","version","country","ip_id"',
 u'"2015-12-12","13:42:10",257886,"3.2.2","i386","mingw32","HistData","0.7-6","CZ",1',
 u'"2015-12-12","13:24:37",1236751,"3.2.2","x86_64","mingw32","RJSONIO","1.3-0","DE",2',
 u'"2015-12-12","13:42:35",2077876,"3.2.2","i386","mingw32","UsingR","2.0-5","CZ",1',
 u'"2015-12-12","13:42:01",266724,"3.2.2","i386","mingw32","gridExtra","2.0.0","CZ",1']

In [12]:
# TODO: use takeSample() for sampling 5 entries
raw_content.takeSample(True,5, 3)

[u'"2015-12-12","10:17:30",36250,"3.2.3","i386","mingw32","highr","0.5.1","IT",3247',
 u'"2015-12-12","05:10:40",21351,"3.1.3","x86_64","mingw32","lazyeval","0.1.10","US",5684',
 u'"2015-12-12","13:06:32",494138,"3.2.3","x86_64","linux-gnu","rjson","0.2.15","KR",655',
 u'"2015-12-12","00:58:48",216079,NA,NA,NA,"mvtnorm","1.0-3","US",231',
 u'"2015-12-12","03:49:15",247550,"3.1.3","x86_64","mingw32","colorspace","1.2-6","US",499']

## 3] Split each row in the raw content RDD by a comma or "," and remove double quotes around the string value from the columns 

In [0]:
# Split each row in the raw content RDD by a comma
content = raw_content.map(lambda x: x.split(','))
content.take(5)

In [0]:
# Remove double quotes around string value from the column
def clean(x):
  return([xx.replace('"', '') for xx in x])

content = content.map(clean)

content.take(5)

## 4] Reduce and Counting: work on the RDD generated in Question-3
### 1. Get the RDD for counting downloading records for each package (7th column): use reduceByKey(). Get how many distinct packages are there? Also print the count for 'dbmss' package.
### 2. Get the RDD for counting downloading records for each package (7th column): use countByKey(). Print the count for 'dbmss' package.
### 3. As countByKey() works only on RDDs of type (K, V), check the effect of the value of V by using different values for V and print the count for 'dbmss' package after each case to check the effect of a value for V.

# 4.1

In [15]:
# Get the number of packages of 7th column using reduceByKey()
package_count = content.map(lambda x: (x[6], 1)).reduceByKey(lambda a,b: a+b)
#print the type
print (type(package_count))
#the total number of distinct packages
package_count.count()
#Take the first five values for package_count
package_count.take(5)


<class 'pyspark.rdd.PipelinedRDD'>


[(u'SIS', 24),
 (u'StatMethRank', 15),
 (u'dbmss', 54),
 (u'searchable', 14),
 (u'RcmdrPlugin.TextMining', 3)]

# 4.2

In [16]:
#Use countByKey to perform the above function
package_count_2 = content.map(lambda x: (x[6], 1)).countByKey()
#print the type for package count using countByKey
print(type(package_count_2))
#Count the number of dbmss
package_count_2['dbmss']


<type 'collections.defaultdict'>


54

# 4.3

Using v =2

In [17]:
#Use countByKey to perform the above function
package_count_3 = content.map(lambda x: (x[6], 2)).countByKey()
#print the type for package count using countByKey
print(type(package_count_3))
#Count the number of dbmss
package_count_3['dbmss']

<type 'collections.defaultdict'>


54

Using v= 5

In [18]:
#Use countByKey to perform the above function
package_count_4 = content.map(lambda x: (x[6], 5)).countByKey()
#print the type for package count using countByKey
print(type(package_count_4))
#Count the number of dbmss
package_count_4['dbmss']

<type 'collections.defaultdict'>


54

## 5] Sorting: Work on the RDD generated in Question 4-1.
### 1. On the package count RDD, use sortByKey() to sort in descending order and print first-5 or 10 entries
### 2. On the package count RDD, use sortByKey() to sort in ascending order and print first-5 or 10 entries
### 3. On the package count RDD, use sortBy() to sort in descending order and print first-5 or 10 entries
### 4. On the package count RDD, use sortBy() to sort in ascending order and print first-5 or 10 entries

# 5.1

In [22]:
# Sort in descending order and print the first 10 entries
package_count.map(lambda x: (x[1], x[0])).sortByKey(0).take(10)


[(4783, u'Rcpp'),
 (3913, u'ggplot2'),
 (3748, u'stringi'),
 (3449, u'stringr'),
 (3436, u'plyr'),
 (3265, u'magrittr'),
 (3223, u'digest'),
 (3205, u'reshape2'),
 (3046, u'RColorBrewer'),
 (3007, u'scales')]

# 5.2

In [23]:
# Sort in ascending order and print the first 10 entries
package_count.map(lambda x: (x[1], x[0])).sortByKey(1).take(10)

[(1, u'parspatstat'),
 (1, u'ssh.utils'),
 (1, u'bscr'),
 (1, u'CVD'),
 (1, u'postgwas'),
 (1, u'mosaicManip'),
 (1, u'rcqp'),
 (1, u'rolasized'),
 (1, u'iScreen'),
 (1, u'GFMaps')]

# 5.3

In [24]:
# sortBy in descending order
package_count.sortBy(lambda x:x[1], ascending = False).take(10)

[(u'Rcpp', 4783),
 (u'ggplot2', 3913),
 (u'stringi', 3748),
 (u'stringr', 3449),
 (u'plyr', 3436),
 (u'magrittr', 3265),
 (u'digest', 3223),
 (u'reshape2', 3205),
 (u'RColorBrewer', 3046),
 (u'scales', 3007)]

# 5.4

In [25]:
# sortBy in ascending order
package_count.sortBy(lambda x:x[1]).take(10)

[(u'parspatstat', 1),
 (u'ssh.utils', 1),
 (u'bscr', 1),
 (u'CVD', 1),
 (u'postgwas', 1),
 (u'mosaicManip', 1),
 (u'rcqp', 1),
 (u'rolasized', 1),
 (u'iScreen', 1),
 (u'GFMaps', 1)]

## 6] Filter: Work on the RDD generated in Question 3; Get the date, time and size for the rows with package is 'Rtts' and country is 'US'

In [29]:
content.filter(lambda x: x[6] == 'Rtts' and x[8] == 'US').count()
content.filter(lambda x: x[6] == 'Rtts' and x[8] == 'US').take(2)


[[u'2015-12-12',
  u'04:52:36',
  u'23820',
  u'3.2.3',
  u'i386',
  u'mingw32',
  u'Rtts',
  u'0.3.3',
  u'US',
  u'1652'],
 [u'2015-12-12',
  u'20:31:45',
  u'23820',
  u'3.2.3',
  u'x86_64',
  u'linux-gnu',
  u'Rtts',
  u'0.3.3',
  u'US',
  u'4438']]

## 7] Set operations: Work on the raw content RDD from Question 1
### 1. Union: union raw content RDD (raw_content) with itself and get the total count after the union operation
### 2. Intersect: get the distinct value set of raw content RDD using intersect operation
### 3. Use the distinct() function to get the count of distinct rows

# 7.1

In [30]:
# Union
raw_content.union(raw_content)
#Get count after the union
raw_content.union(raw_content).count()


843940

# 7.2

In [31]:
# Intersection
raw_content.intersection(raw_content)
# Get the distinct counts after intersection 
raw_content.intersection(raw_content).count()

421553

# **7.3**

In [32]:
# Distinct values
raw_content.distinct().count()

421553

## 8] Join: Work on the RDD generated in Question 3
### 1. Generate a new RDD in which the values from the 'country' column is a KEY
### 2. For a given mapping of the abbreviates of four countries and their full name perform join and left outer join using join() and leftOuterJoin() functions. Also sample 8 entries from each.

# Set columns containing country x[8] as key

In [0]:
content2=content.map(lambda x:(x[8], x))

Give a mapping table of the four countries and their full names

In [0]:
mapping=[('DE', 'Germany'), ('US', 'United States'), ('CN', 'China'), ('IN',"India")]
mapping=sc.parallelize(mapping)

# Work on Question 8


# **8.1**

In [35]:
# join and sample 8 entries
content2.join(mapping).takeSample(False, 8)

[(u'CN',
  ([u'2015-12-12',
    u'22:25:27',
    u'514',
    u'NA',
    u'NA',
    u'NA',
    u'SightabilityModel',
    u'1.3',
    u'CN',
    u'2365'],
   'China')),
 (u'US',
  ([u'2015-12-12',
    u'15:43:21',
    u'35212',
    u'3.1.1',
    u'x86_64',
    u'linux-gnu',
    u'reshape2',
    u'1.4.1',
    u'US',
    u'8987'],
   'United States')),
 (u'CN',
  ([u'2015-12-12',
    u'23:06:50',
    u'514',
    u'NA',
    u'NA',
    u'NA',
    u'fmri',
    u'1.5-1',
    u'CN',
    u'4692'],
   'China')),
 (u'CN',
  ([u'2015-12-12',
    u'21:52:59',
    u'511',
    u'NA',
    u'NA',
    u'NA',
    u'SPA3G',
    u'1.0',
    u'CN',
    u'8154'],
   'China')),
 (u'CN',
  ([u'2015-12-12',
    u'21:27:33',
    u'511',
    u'NA',
    u'NA',
    u'NA',
    u'TimeProjection',
    u'0.2.0',
    u'CN',
    u'67'],
   'China')),
 (u'CN',
  ([u'2015-12-12',
    u'20:12:13',
    u'97194',
    u'3.2.2',
    u'i386',
    u'mingw32',
    u'rsem',
    u'0.4.6',
    u'CN',
    u'41'],
   'China')),
 (u'CN',

# 8.2

In [36]:
content2.leftOuterJoin(mapping).takeSample(False, 8)

[(u'CN',
  ([u'2015-12-12',
    u'23:12:34',
    u'512',
    u'NA',
    u'NA',
    u'NA',
    u'multispatialCCM',
    u'1.0',
    u'CN',
    u'4536'],
   'China')),
 (u'GB',
  ([u'2015-12-12',
    u'12:43:50',
    u'5487738',
    u'3.2.3',
    u'i386',
    u'mingw32',
    u'Morpho',
    u'2.3.0',
    u'GB',
    u'548'],
   None)),
 (u'US',
  ([u'2015-12-12',
    u'05:09:11',
    u'75326',
    u'3.2.3',
    u'x86_64',
    u'linux-gnu',
    u'xml2',
    u'0.1.2',
    u'US',
    u'5484'],
   'United States')),
 (u'US',
  ([u'2015-12-12',
    u'05:10:25',
    u'1148897',
    u'3.2.3',
    u'i386',
    u'mingw32',
    u'RJSONIO',
    u'1.3-0',
    u'US',
    u'5658'],
   'United States')),
 (u'CN',
  ([u'2015-12-12',
    u'03:25:52',
    u'201469',
    u'3.2.2',
    u'x86_64',
    u'mingw32',
    u'MAc',
    u'1.1',
    u'CN',
    u'7506'],
   'China')),
 (u'KR',
  ([u'2015-12-12',
    u'16:32:51',
    u'155907',
    u'3.2.3',
    u'x86_64',
    u'mingw32',
    u'pgam',
    u'0.4.12',
    u